# PyTorch Computer vision

## 1.Computer vision libraries

* torchvision - base domain library
* torchvision.datasets
* torchvision.models - get pretrained models
* torchvision.transformers
* torch.utils.data.Dataset
* torch.utils.data.DataLoader



In [ ]:
import torch
from torch import nn
#Torch Vison
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
print(torch.__version__)
print(torchvision.__version__)

##1.Getting a Dataset
>  MNIST


In [ ]:
#Setup training data
train_data = datasets.MNIST(
    root = 'data',
    train =True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)
test_data =  datasets.MNIST(
    root = 'data',
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
len(train_data), len(test_data)

In [ ]:
image, label = train_data[0]
image, label

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
train_data.targets

In [ ]:
print(f'Image shape={image.shape}, Labels ={label}')

In [ ]:
#Visualize
import matplotlib.pyplot as plt
image ,label = train_data[0]
plt.title(class_names[label])
plt.imshow(image.squeeze(), cmap='hot')
plt.axis(False)

In [ ]:
#Plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols =4, 4
for i in range(1, rows*cols+1):
  rand = torch.randint(0,len(train_data), size=[1]).item()
  img, label = train_data[rand]
  fig.add_subplot(rows,cols, i)
  plt.imshow(img.squeeze(), cmap='hot')
  plt.title(class_names[label])
  plt.axis(False)

# 2.Prepare Data Loader

In [ ]:
train_data,test_data

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
from torch.utils.data import DataLoader

#Set up batch size hyperparameter
batch = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=batch,
                            shuffle=True)

test_dataloader = DataLoader(dataset= test_data,
                             batch_size=batch,
                             shuffle=False)

train_dataloader,test_dataloader

In [ ]:
#Check what is creaked
print(f'DataLoaders: {train_dataloader, test_dataloader}')
print(f'Lenght of train: {len(train_dataloader)} batches of {batch}')
print(f'Lenght of train: {len(test_dataloader)} batches of {batch}')

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
#Visualizing a single peice of a dataloader
torch.manual_seed(42)
random_idx = torch.randint(0,len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap='hot')
plt.title(class_names[label])
plt.axis(False)
print(f'Image size: {img.shape}')
print(f'label: {label}, label size: {label.shape}')

#3.Model building

beginning from model 0

In [ ]:
flatten_model = nn.Flatten()

x=train_features_batch[0]

output = flatten_model(x)
print(output.shape)

In [ ]:
import torch
from torch import nn

class FashionMNISTModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
next(model_0.parameters()).device


In [ ]:
torch.manual_seed(42)

model_0 = FashionMNISTModelV0()
model_0.to(device)
model_0


In [ ]:
dummy_x = torch.rand([1, 1, 28, 28]).to(device)
model_0(dummy_x)


In [ ]:
model_0.state_dict()

In [ ]:
import requests
from pathlib import Path
#Download helper functions
if Path('helper_functions.py').is_file():
  print('File already exists, skipping download')
else:
  print('Downloading file')
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/af6548e52663c8395fe2777bd0fbfd6ac85fd6f8/helper_functions.py")
  with open('helper_functions.py','wb') as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_0.parameters(),
                            lr=0.1)


In [ ]:
#Creating a function to time our experiments
from timeit import default_timer as timer
def print_train_time(start:float,
                     end: float,
                     device: torch.device = None):
  total_time = end-start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [ ]:
start_time =timer()
end_time=timer()
print_train_time(start=start_time, end=end_time, device= 'cpu')

In [ ]:
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n-----------')
    train_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)  # <-- move to device

        model_0.train()
        y_pred = model_0(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            print(f'Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples')

    train_loss /= len(train_dataloader)

    # Evaluation
    test_loss, test_acc = 0, 0
    model_0.eval()
    with torch.inference_mode():
      for X, y in test_dataloader:
          X, y = X.to(device), y.to(device)
          test_pred = model_0(X)
          test_loss += loss_fn(test_pred, y).item()
          test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))



    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
    print(f'Train loss: {train_loss:.5f} | Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}')


#4. Make predictions and get results

In [ ]:
from tqdm import tqdm
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for x, y in tqdm(data_loader):
            x, y = x.to(device), y.to(device)  # <-- move to device
            y_pred = model(x)
            loss += loss_fn(y_pred, y).item()
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

        loss /= len(data_loader)
        acc /= len(data_loader)

    return {
    'Model Name': model.__class__.__name__,
    'Model Loss': loss,
    'Model Accuracy': acc
}



model_0_results = eval_model(model=model_0,
                             data_loader=test_dataloader,
                             loss_fn=loss_fn,
                             accuracy_fn=accuracy_fn)
model_0_results

In [ ]:
!pip install Pillow
from PIL import ImageOps

In [ ]:
import torch
from PIL import Image, ImageOps
from torchvision import transforms
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_0.to(device)

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

def predict_image(image_path):
    image = Image.open(image_path).convert('L')
    image = ImageOps.invert(image)
    image = transform(image).unsqueeze(0).to(device)
    model_0.eval()
    with torch.no_grad():
        output = model_0(image)
        _, predicted_class = torch.max(output, 1)
    return predicted_class.item()

image_path = '/content/1.jpg'
predicted_class = predict_image(image_path)
print(f'Predicted Class: {predicted_class}')
print(f'Class Name: {class_names[predicted_class]}')

def show_tensor_image(tensor_img):
    img = tensor_img.squeeze().cpu().numpy()
    plt.imshow(img, cmap='hot')
    plt.axis('off')
    plt.title("Preprocessed Image")
    plt.show()

image_tensor = transform(ImageOps.invert(Image.open(image_path).convert('L')))
show_tensor_image(image_tensor)
